In [ ]:
######################################################################################
# Using a Policy Gradient Model to beat the CPU at the PLE pong environment
# Implementing the REINFORCE algorithm (Monte Carlo policy gradient)

# Need to: 
# - Implement policy gradient 
# - Add reward-to-go
# - Add discounting 
# - Add baseline 

######################################################################################

In [ ]:
from ple.games.pong import Pong
from ple import PLE
import pygame
from pygame.constants import K_w, K_s
import numpy as np
import import_ipynb

from Policy_Gradient_Model import *
from Utils import *

##############################
# VARIABLES 
RENDER = True
MAX_SCORE = 1
NB_FRAMES = 10000
EPISODES = 10
##############################

policy_model = agent()

for e in range(EPISODES):
    
    pygame.init()
    game = Pong(width=512, height=400, MAX_SCORE=MAX_SCORE)
    game.screen = pygame.display.set_mode(game.getScreenDims(), 0, 32)
    env = PLE(game, fps=30, display_screen=RENDER, force_fps=False, reward_values = {"tick": 0.1, "loss": -5.0, "win": 5.0})
    env.init()
    
    # get the possible actions
    possible_actions = game.actions 
    action_names = ['up', 'down']
    
    state = process_state(env.getGameState())
    total_reward = 0
    rewards = []
    states = []
    actions = []
    
    # for some reason breaks without this    
    env.act(env.NOOP)
    reward = 0.0

    for f in range(NB_FRAMES):           
        
        action = policy_model.act(state) 
        reward = env.act(possible_actions[action_names[action]])
        next_state = process_state(env.getGameState()) 
        
        
        rewards.append(reward)
        states.append(state)
        actions.append(action)
        
        state = next_state
        total_reward += reward
        
        #check if the game is over
        if env.game_over():            
            pygame.quit()
            
            # TODO: fix the following function
            # Problem seems to be related to the model predicting probability values that are too small
            # Made worse by removing standardisation of states
            # Perhaps standardising the states at the end would work most effectively? 
            
            policy_model.train(states, rewards, actions)            
            print("total reward after {} episodes is {}".format(e + 1, total_reward))
        
            break    
            
            
        